In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup
import re
import json
import requests
import openpyxl
import urllib
from urllib import parse
import lxml
from sklearn.model_selection import train_test_split

from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

import time
from sklearn.preprocessing import MinMaxScaler

In [45]:
def normalize_scale(inp):# 환율데이터 표본적어서 *10
    result = []
    for i in inp:
        result.append(((i-np.mean(inp))/np.var(inp))*10)
    return result
def normalize(inp):# *10안함
    result = []
    for i in inp:
        result.append(((i-np.mean(inp))/np.var(inp)))
    return result

class Stock:
    
    Exchange='./Data/Exchange.csv' #객체가 공통으로 사용할 환율 데이터 (동적크롤링할줄몰라서 csv로 받아옴)
    
    def __init__(self,code):
        self.code=code
    
    @staticmethod
    def exchangeRate():  ##1달간격의 환율의 종가데이터  1행배열 57요소( 18.05.01~  23.01.01) 배열로 출력 SIZE(1,57)
        csv = pd.read_csv(Stock.Exchange,encoding='utf8')
        #print(csv)  환율 전체데이터 확인하기            
        csv = np.array(csv)
        data=[]
        for i in range(len(csv)):   
            fnum=""
            for j in csv[i][1]:
                if(j == ','):
                    continue
                fnum+=j
            data.append(float(fnum))
        data = np.array(data)
        data = data.astype('float32')
        data = normalize_scale(data) 
        return data

    def idxNorm(self):##ROA, ROE, EPS, BPS, DPS, PER, PBR의 각 항목당 (2018.12 ~ 2023.12)의 데이터 6개 갖음 SIZE(7,8)
        get_param = {
            'pGB':1,
            'gicode':'A%s'%(self.code),
            'cID':'',
            'MenuYn':'Y',
            'ReportGB':'',
            'NewMenuID':101,
            'stkGb':701,
        }
        get_param = parse.urlencode(get_param)
        url="http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?%s"%(get_param)
        tables = pd.read_html(url, header=0,encoding='utf-8')
        sit=np.array(tables[11]) 

        data=[]#정규화 시작

        for i in range(7):
            crw=[]
            if (i == 4): continue
            for j in range(7):
                
                if(sit[17+i][j+1] != sit[17+i][j+1]):
                    idx =  (float(sit[17+i][j]) + float(sit[17+i][j+2]))/2.0
                    crw.append(idx)
                    continue
                idx = float(sit[17+i][j+1])
                crw.append(idx)
            data.append(normalize(crw))
        data=np.array(data)
        return data
    
    def priceList(self):
        csv = pd.read_csv('./Data/주식가격/%s.csv'%(self.code))
        cut_csv = csv[:][:1148]
        arr = np.array(cut_csv)
        array = arr[:,1]
        array = array/10000
        array = np.array(array)
        arr = array.reshape(-1,1)
        temp = [[0] * len(arr[0]) for _ in range(len(arr))]
        for i in range(len(arr)):
            temp[i] = arr[len(arr)-1-i]
        temp = np.array(temp).astype(float)
        return temp
    
    def amount(self): #거래량 정규화 된 함수
        csv = pd.read_csv('./Data/주식가격/%s.csv'%(self.code))
        cut_csv = csv[:][:1148]
        arr = np.array(cut_csv)
        array = arr[:,6]
        n=int(np.mean(array))
        count=1
        while(n != 0):
            n = int(n/10);
            if n == 0 : break
            count=count*10;
        array = arr[:,6]/count
        array = np.array(array)
        arr = array.reshape(-1,1)
        temp = [[0] * len(arr[0]) for _ in range(len(arr))]
        for i in range(len(arr)):
            temp[i] = arr[len(arr)-1-i]
        temp = np.array(temp).astype(float)
        return temp
    def priceamount(self):
        return np.concatenate((self.priceList(), self.amount()), axis=1)
    def showChart(self): ##종목코드의 제무재표를 나타내는 함수
        get_param = {
            'pGB':1,
            'gicode':'A%s'%(self.code),
            'cID':'',
            'MenuYn':'Y',
            'ReportGB':'',
            'NewMenuID':101,
            'stkGb':701,
        }
        get_param = parse.urlencode(get_param)
        url="http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?%s"%(get_param)
        tables = pd.read_html(url, header=0,encoding='utf-8')
        return tables[11]


In [74]:
def concatData(a, data):
    a2023 = a[1147:]
    a2022 = a[901:1147]
    a2021 = a[653:901]
    a2020 = a[405:653]
    a2019 = a[159:405]
    a2018 = a[0:159]
    
    data2023 = np.full((len(a2023),6), data[:,5])
    data2022 = np.full((len(a2022),6), data[:,4])
    data2021 = np.full((len(a2021),6), data[:,3])
    data2020 = np.full((len(a2020),6), data[:,2])
    data2019 = np.full((len(a2019),6), data[:,1])
    data2018 = np.full((len(a2018),6), data[:,0])
    
    data1 = data2023
    data1 = np.concatenate((np.full((len(a2023),6), data[:,5]),
                            np.full((len(a2022),6), data[:,4]),
                            np.full((len(a2021),6), data[:,3]),
                            np.full((len(a2020),6), data[:,2]),
                            np.full((len(a2019),6), data[:,1]),
                            np.full((len(a2018),6), data[:,0])), axis=0)
    arr = np.concatenate((a, data1), axis=1)
    array = np.array(arr)
    
    er = Stock.exchangeRate()
    er = np.array(er).reshape(-1, 1)
    
    exchange = np.full((1,1), er[-57])
    exchange = np.concatenate((exchange, np.full((21,1), er[-56]),
                               np.full((22,1), er[-55]),
                               np.full((19,1), er[-54]),
                               np.full((20,1), er[-53]),
                               np.full((22,1), er[-52]),
                               np.full((21,1), er[-51]),
                               np.full((20,1), er[-50]),
                               np.full((21,1), er[-49]),
                               np.full((21,1), er[-48]),
                               np.full((21,1), er[-47]),
                               np.full((18,1), er[-46]),
                               np.full((20,1), er[-45]),
                               np.full((22,1), er[-44]),
                               np.full((22,1), er[-43]),
                               np.full((19,1), er[-42]),
                               np.full((19,1), er[-41]),
                               np.full((21,1), er[-40]),
                               np.full((22,1), er[-39]),
                               np.full((22,1), er[-38]),
                               np.full((19,1), er[-37]),
                               np.full((22,1), er[-36]),
                               np.full((22,1), er[-35]),
                               np.full((18,1), er[-34]),
                               np.full((20,1), er[-33]),
                               np.full((21,1), er[-32]),
                               np.full((21,1), er[-31]),
                               np.full((19,1), er[-30]),
                               np.full((21,1), er[-29]),
                               np.full((20,1), er[-28]),
                               np.full((23,1), er[-27]),
                               np.full((22,1), er[-26]),
                               np.full((19,1), er[-25]),
                               np.full((20,1), er[-24]),
                               np.full((22,1), er[-23]),
                               np.full((20,1), er[-22]),
                               np.full((20,1), er[-21]),
                               np.full((20,1), er[-20]),
                               np.full((21,1), er[-19]),
                               np.full((21,1), er[-18]),
                               np.full((19,1), er[-17]),
                               np.full((21,1), er[-16]),
                               np.full((23,1), er[-15]),
                               np.full((19,1), er[-14]),
                               np.full((21,1), er[-13]),
                               np.full((22,1), er[-12]),
                               np.full((20,1), er[-11]),
                               np.full((17,1), er[-10]),
                               np.full((22,1), er[-9]),
                               np.full((19,1), er[-8]),
                               np.full((22,1), er[-7]),
                               np.full((21,1), er[-6]),
                               np.full((17,1), er[-5]),
                               np.full((22,1), er[-4]),
                               np.full((22,1), er[-3]),
                               np.full((19,1), er[-2]),
                               np.full((17,1), er[-1])), axis=0)
    train_inputs = np.concatenate((array, exchange), axis=1)
    return train_inputs
def returnInputs(a,b):
    inputs = concatData(a, b)
    results = inputs[:-1,:].astype(float)
    return results
def returnDailyLabels(a,b):
    inputs = concatData(a, b)
    results = inputs[1:,0].reshape(-1,1).astype(float)
    return results
def returnWeeklyLabels(a,b):
    results = []
    inputs = concatData(a, b)
    for i in range(1, 1142):
        results.append((inputs[i:i+7,0]).astype(float))
    results = np.array(results)
    return results
def returnMonthlyLabels(a,b):
    results = []
    inputs = concatData(a, b)
    for i in range(1, 1119):
        results.append((inputs[i:i+30,0]).astype(float))
    results = np.array(results)
    return results
def reverseData(arr):
    temp = [[0] * len(arr[0]) for _ in range(len(arr))]
    for i in range(len(arr)):
        temp[i] = arr[len(arr)-1-i]
    return temp
def scaleData(a):
    scaler = MinMaxScaler()
    scale_cols = [0]
    df_scaled = scaler.fit_transform(a[scale_cols])

In [46]:
SS = Stock('005930')
SK = Stock('000660')
DW=Stock('049770')
SFA=Stock('056190')
GB=Stock('024110')

sa=SS.priceamount()
sk=SK.priceamount()
dw=DW.priceamount()
sf=SFA.priceamount()
gb=GB.priceamount()
sadata=SS.idxNorm()
skdata=SK.idxNorm()
dwdata=DW.idxNorm()
sfdata=SFA.idxNorm()
gbdata=GB.idxNorm()

C:\Users\wjddu\AppData\Local\Temp\ipykernel_12128\1354609297.py:70: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  csv = pd.read_csv('./Data/주식가격/%s.csv'%(self.code))
C:\Users\wjddu\AppData\Local\Temp\ipykernel_12128\1354609297.py:84: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  csv = pd.read_csv('./Data/주식가격/%s.csv'%(self.code))
C:\Users\wjddu\AppData\Local\Temp\ipykernel_12128\1354609297.py:70: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  csv = pd.read_csv('./Data/주식가격/%s.csv'%(self.code))
C:\Users\wjddu\AppData\Local\Temp\ipykernel_12128\1354609297.py:84: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  csv = pd.read_csv('./Data/주식가격/%s.csv'%(self.code))


In [72]:
dtrain_inputs = []
dtrain_labels = []
dtrain_inputs.append(returnInputs(sa, sadata))
dtrain_inputs.append(returnInputs(sk, skdata))
dtrain_inputs.append(returnInputs(dw, dwdata))
dtrain_inputs.append(returnInputs(sf, sfdata))
dtrain_inputs.append(returnInputs(gb, gbdata))
dtrain_inputs = np.array(dtrain_inputs)

dtrain_labels.append(returnDailyLabels(sa, sadata))
dtrain_labels.append(returnDailyLabels(sk, skdata))
dtrain_labels.append(returnDailyLabels(dw, dwdata))
dtrain_labels.append(returnDailyLabels(sf, sfdata))
dtrain_labels.append(returnDailyLabels(gb, gbdata))
dtrain_labels = np.array(dtrain_labels)
print(dtrain_labels, dtrain_labels.shape)

[[[ 5.09 ]
  [ 5.16 ]
  [ 5.13 ]
  ...
  [ 5.66 ]
  [ 5.53 ]
  [ 5.55 ]]

 [[ 8.34 ]
  [ 8.58 ]
  [ 8.61 ]
  ...
  [ 7.6  ]
  [ 7.5  ]
  [ 7.57 ]]

 [[24.75 ]
  [24.5  ]
  [24.05 ]
  ...
  [16.   ]
  [15.7  ]
  [15.6  ]]

 [[ 3.475]
  [ 3.385]
  [ 3.52 ]
  ...
  [ 3.74 ]
  [ 3.64 ]
  [ 3.53 ]]

 [[ 1.675]
  [ 1.675]
  [ 1.695]
  ...
  [ 1.   ]
  [ 0.982]
  [ 0.946]]] (5, 1147, 1)


In [92]:
wtrain_inputs = []
wtrain_labels = []
wtrain_inputs.append(returnInputs(sa, sadata))
wtrain_inputs.append(returnInputs(sk, skdata))
wtrain_inputs.append(returnInputs(dw, dwdata))
wtrain_inputs.append(returnInputs(sf, sfdata))
wtrain_inputs.append(returnInputs(gb, gbdata))
wtrain_inputs = np.array(wtrain_inputs)
wtrain_inputs = wtrain_inputs[:,:1141,:]

wtrain_labels.append(returnWeeklyLabels(sa, sadata))
wtrain_labels.append(returnWeeklyLabels(sk, skdata))
wtrain_labels.append(returnWeeklyLabels(dw, dwdata))
wtrain_labels.append(returnWeeklyLabels(sf, sfdata))
wtrain_labels.append(returnWeeklyLabels(gb, gbdata))
wtrain_labels = np.array(wtrain_labels)
wtrain_labels = wtrain_labels[:,-1,:]
print(wtrain_labels.shape, wtrain_labels)

(5, 7) [[ 5.91   5.81   5.79   5.81   5.66   5.53   5.55 ]
 [ 7.92   7.78   7.7    7.7    7.6    7.5    7.57 ]
 [16.15  16.1   16.1   16.05  16.    15.7   15.6  ]
 [ 3.77   3.78   3.83   3.835  3.74   3.64   3.53 ]
 [ 1.11   1.11   1.105  1.12   1.     0.982  0.946]]


In [93]:
mtrain_inputs = []
mtrain_labels = []
mtrain_inputs.append(returnInputs(sa, sadata))
mtrain_inputs.append(returnInputs(sk, skdata))
mtrain_inputs.append(returnInputs(dw, dwdata))
mtrain_inputs.append(returnInputs(sf, sfdata))
mtrain_inputs.append(returnInputs(gb, gbdata))
mtrain_inputs = np.array(mtrain_inputs)
mtrain_inputs = mtrain_inputs[:,:1118,:]

mtrain_labels.append(returnMonthlyLabels(sa, sadata))
mtrain_labels.append(returnMonthlyLabels(sk, skdata))
mtrain_labels.append(returnMonthlyLabels(dw, dwdata))
mtrain_labels.append(returnMonthlyLabels(sf, sfdata))
mtrain_labels.append(returnMonthlyLabels(gb, gbdata))
mtrain_labels = np.array(mtrain_labels)
mtrain_labels = mtrain_labels[:,-1,:]
print(mtrain_labels.shape, mtrain_labels)

(5, 30) [[ 6.14   6.06   6.1    6.14   6.1    6.01   6.06   6.22   6.26   6.04
   6.03   5.92   5.89   5.92   6.04   5.95   5.97   6.05   5.93   5.95
   5.95   5.86   5.8    5.91   5.81   5.79   5.81   5.66   5.53   5.55 ]
 [ 8.63   8.52   8.52   8.69   8.51   8.31   8.37   8.5    8.47   8.19
   8.1    8.1    7.89   7.88   8.15   8.11   8.23   8.18   8.02   7.84
   7.9    7.83   7.8    7.92   7.78   7.7    7.7    7.6    7.5    7.57 ]
 [13.85  13.95  14.05  14.    14.05  13.6   13.7   13.85  14.05  14.25
  14.2   14.2   14.15  14.25  14.4   14.7   14.55  14.65  14.7   14.75
  14.75  14.65  15.65  16.15  16.1   16.1   16.05  16.    15.7   15.6  ]
 [ 3.9    3.97   4.07   4.005  3.97   3.905  3.975  4.     3.935  3.885
   3.905  3.805  3.785  3.775  3.82   3.69   3.71   3.765  3.755  3.685
   3.695  3.61   3.635  3.77   3.78   3.83   3.835  3.74   3.64   3.53 ]
 [ 1.09   1.1    1.095  1.105  1.105  1.095  1.105  1.12   1.12   1.12
   1.125  1.11   1.09   1.085  1.105  1.105  1.11   1.11   

In [77]:
dmodel = Sequential([
    layers.LSTM(512, input_shape=(dtrain_inputs.shape[1],dtrain_inputs.shape[2]), return_sequences=True),
    layers.LSTM(128),
    layers.Dense(1, activation = 'relu')
])

dmodel.compile(loss='mse',optimizer='adam', metrics='mae')
dmodel.summary()
dmodel.fit(dtrain_inputs, dtrain_labels,
    epochs=10
)

Model: "sequential_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_56 (LSTM)              (None, 1147, 512)         1069056   
                                                                 
 lstm_57 (LSTM)              (None, 128)               328192    
                                                                 
 dense_28 (Dense)            (None, 1)                 129       
                                                                 
Total params: 1,397,377
Trainable params: 1,397,377
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
1/1 [==============================] - 8s 8s/step - loss: 120.8692 - mae: 8.1955
Epoch 2/10
1/1 [==============================] - 2s 2s/step - loss: 80.1699 - mae: 5.8760
Epoch 3/10
1/1 [==============================] - 2s 2s/step - loss: 61.5050 - mae: 4.7062
Epoch 4/10
1/1 [=================

In [98]:
wmodel = Sequential([
    layers.LSTM(512, input_shape=(wtrain_inputs.shape[1],wtrain_inputs.shape[2]), return_sequences=True),
    layers.LSTM(128),
    layers.Dense(7, activation = 'relu')
])

wmodel.compile(loss='mse',optimizer='adam')
wmodel.summary()
wmodel.fit(wtrain_inputs, wtrain_labels,
    epochs=100
)

Model: "sequential_36"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_76 (LSTM)              (None, 1141, 512)         1069056   
                                                                 
 lstm_77 (LSTM)              (None, 128)               328192    
                                                                 
 dense_38 (Dense)            (None, 7)                 903       
                                                                 
Total params: 1,398,151
Trainable params: 1,398,151
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
1/1 [==============================] - 9s 9s/step - loss: 70.1318
Epoch 2/100
1/1 [==============================] - 2s 2s/step - loss: 59.9663
Epoch 3/100
1/1 [==============================] - 2s 2s/step - loss: 52.1950
Epoch 4/100
1/1 [==============================] - 2s 2s/step - loss: 45.

1/1 [==============================] - 2s 2s/step - loss: 2.4760
Epoch 95/100
1/1 [==============================] - 2s 2s/step - loss: 2.3789
Epoch 96/100
1/1 [==============================] - 2s 2s/step - loss: 2.2864
Epoch 97/100
1/1 [==============================] - 2s 2s/step - loss: 2.1909
Epoch 98/100
1/1 [==============================] - 2s 2s/step - loss: 2.1003
Epoch 99/100
1/1 [==============================] - 2s 2s/step - loss: 2.0203
Epoch 100/100
1/1 [==============================] - 2s 2s/step - loss: 1.9335


In [101]:
a=[]
a.append(returnInputs(sa, sadata))
a = np.array(a)
a = a[:,:1141,:]
print(a)
wmodel.predict(a)

[[[ 5.26        2.310472   -0.14501075 ...  0.07794622 -0.56552767
    0.15273633]
  [ 5.09        1.6128305   0.06160386 ... -0.06145953  2.42886884
    0.13651005]
  [ 5.16        1.3905263   0.06160386 ... -0.06145953  2.42886884
    0.13651005]
  ...
  [ 5.95        0.7696187   0.42443926 ... -0.20792381  2.02284898
   -0.18861003]
  [ 5.86        0.9284761   0.42443926 ... -0.20792381  2.02284898
   -0.18861003]
  [ 5.8         1.0356971   0.42443926 ... -0.20792381  2.02284898
   -0.18861003]]]
1/1 [==============================] - 0s 417ms/step


array([[5.7175627, 5.766752 , 5.7722607, 5.8064857, 5.7478204, 5.681718 ,
        5.64089  ]], dtype=float32)

In [85]:
mmodel = Sequential([
    layers.LSTM(512, input_shape=(mtrain_inputs.shape[1],mtrain_inputs.shape[2]), return_sequences=True),
    layers.LSTM(128),
    layers.Dense(30, activation = 'relu')
])

mmodel.compile(loss='mse',optimizer='adam')
mmodel.summary()
mmodel.fit(mtrain_inputs, mtrain_labels,
    epochs=100
)

Model: "sequential_34"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_72 (LSTM)              (None, 1118, 512)         1069056   
                                                                 
 lstm_73 (LSTM)              (None, 128)               328192    
                                                                 
 dense_36 (Dense)            (None, 30)                3870      
                                                                 
Total params: 1,401,118
Trainable params: 1,401,118
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100


InvalidArgumentError: Graph execution error:

Detected at node 'mean_squared_error/SquaredDifference' defined at (most recent call last):
    File "C:\Users\wjddu\Anaconda3\envs\jybgpu\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\wjddu\Anaconda3\envs\jybgpu\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\wjddu\Anaconda3\envs\jybgpu\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\wjddu\Anaconda3\envs\jybgpu\lib\site-packages\traitlets\config\application.py", line 982, in launch_instance
      app.start()
    File "C:\Users\wjddu\Anaconda3\envs\jybgpu\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "C:\Users\wjddu\Anaconda3\envs\jybgpu\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\wjddu\Ana

In [ ]:
dmodel.save('dailymodel.h5')
wmodel.save('weeklymodel.h5')
mmodel.save('monthlymodel.h5')

In [23]:
a=[]
a.append(reverseData(returnInputs(sa, sadata)))
a = np.array(a)
print(a)
wmodel.predict(a)

[[[ 5.53        0.42443926  0.29290524 ... -0.20792381  2.02284898
   -0.18861003]
  [ 5.66        0.42443926  0.29290524 ... -0.20792381  2.02284898
   -0.18861003]
  [ 5.81        0.42443926  0.29290524 ... -0.20792381  2.02284898
   -0.18861003]
  ...
  [ 5.16        0.06160386  0.0435162  ... -0.06145953  2.42886884
    0.13651005]
  [ 5.09        0.06160386  0.0435162  ... -0.06145953  2.42886884
    0.13651005]
  [ 5.26       -0.14501075 -0.10070031 ...  0.07794622 -0.56552767
    0.15273633]]]


ValueError: in user code:

    File "C:\Users\wjddu\Anaconda3\envs\jybgpu\lib\site-packages\keras\engine\training.py", line 1845, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\wjddu\Anaconda3\envs\jybgpu\lib\site-packages\keras\engine\training.py", line 1834, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\wjddu\Anaconda3\envs\jybgpu\lib\site-packages\keras\engine\training.py", line 1823, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\wjddu\Anaconda3\envs\jybgpu\lib\site-packages\keras\engine\training.py", line 1791, in predict_step
        return self(x, training=False)
    File "C:\Users\wjddu\Anaconda3\envs\jybgpu\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\wjddu\Anaconda3\envs\jybgpu\lib\site-packages\keras\engine\input_spec.py", line 264, in a